<a href="https://colab.research.google.com/github/satankita/AmazonReviewsSentimentAnalysis/blob/main/echen52_Copy_of_John's_copy_of_Sentiment_Analysisipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive/", force_remount=True)

mypath = "drive/MyDrive/"
# os.listdir(mypath)

Mounted at /content/drive/


In [ ]:
DATA_DIR = mypath + "TIS Project/Data/"

AMAZON_DATA_TRAIN = "train.csv"
AMAZON_DATA_TEST = "test.csv"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean(text):
    a=[f for f in text if f not in string.punctuation]
    a=''.join(a)
    b=[w for w in a.split() if w.lower() not in stopwords.words('english')]
    return b

# Raw Data Check #

In [ ]:
large_data = pd.read_csv(DATA_DIR + AMAZON_DATA_TRAIN)
data = large_data.sample(frac=0.01, random_state=101)
data.columns = ['Sentiment', 'Review Title', 'Text']
data.drop('Review Title', axis=1, inplace=True)

data['Length'] = data['Text'].apply(len)
data['Word Count'] = data['Text'].apply(lambda x: len(x.split()))

In [ ]:
data.head()

,Sentiment,Text,Length,Word Count
750068,2,"These splices are easy to pinch shut, but do n...",300,55
3084052,2,my prayers answered! a great bra! it is hard t...,113,23
1782585,1,I ordered this watch from Africa and had it co...,270,55
1315001,1,Sound quality for Widescreen Dolby 5.1 version...,404,74
2648609,2,My very first hand of real money Texas Holdem ...,778,148


In [ ]:
data['Sentiment'].value_counts()

,count
Sentiment,
2,18116
1,17884


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data['Text'],data['Sentiment'], test_size=0.3, random_state=101)

In [ ]:
y_train.value_counts()

,count
Sentiment,
2,12629
1,12571


In [ ]:
y_test.value_counts()

,count
Sentiment,
2,5487
1,5313


# VADER Features

In [ ]:
from scipy.sparse import hstack
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC


#large_data = pd.read_csv(DATA_DIR + AMAZON_DATA_TRAIN)
#data = large_data.sample(frac=0.01, random_state=101)
#data.columns = ['Sentiment', 'Review Title', 'Text']
#data.drop('Review Title', axis=1, inplace=True)

#data['Length'] = data['Text'].apply(len)
#data['Word Count'] = data['Text'].apply(lambda x: len(x.split()))

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
def get_vader_sentiment(text):
    return sia.polarity_scores(text)

data['VADER_Sentiment'] = data['Text'].apply(get_vader_sentiment)
data['VADER_Positive'] = data['VADER_Sentiment'].apply(lambda x: x['pos'])
data['VADER_Neutral'] = data['VADER_Sentiment'].apply(lambda x: x['neu'])
data['VADER_Negative'] = data['VADER_Sentiment'].apply(lambda x: x['neg'])
data['VADER_Compound'] = data['VADER_Sentiment'].apply(lambda x: x['compound'])
data.drop(columns=['VADER_Sentiment'], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(
    data[['Text', 'VADER_Positive', 'VADER_Neutral', 'VADER_Negative', 'VADER_Compound']],
    data['Sentiment'],
    test_size=0.3,
    random_state=101
)

cv = CountVectorizer(analyzer=clean)
x_train_text = cv.fit_transform(x_train['Text'])
x_test_text = cv.transform(x_test['Text'])

x_train_vader = x_train[['VADER_Positive', 'VADER_Neutral', 'VADER_Negative', 'VADER_Compound']].values
x_test_vader = x_test[['VADER_Positive', 'VADER_Neutral', 'VADER_Negative', 'VADER_Compound']].values

x_train_combined = hstack([x_train_text, x_train_vader])
x_test_combined = hstack([x_test_text, x_test_vader])

tfidf = TfidfTransformer()
x_train_combined = tfidf.fit_transform(x_train_combined)
x_test_combined = tfidf.transform(x_test_combined)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
checkCV=data['Text'].head(1).apply(clean)
print(checkCV)

750068    [splices, easy, pinch, shut, place, much, pres...
Name: Text, dtype: object


In [ ]:
x_train_text.shape, x_train_vader.shape, x_train_combined.shape

((25200, 89348), (25200, 4), (25200, 89352))

In [ ]:
svm = LinearSVC()
clf = CalibratedClassifierCV(svm)
clf.fit(x_train_combined, y_train)

y_predict = clf.predict(x_test_combined)

print("############# Classification Report ##############")
print(classification_report(y_test, y_predict))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test, y_predict))
print("##################################################")

# BERT Embedding Features

In [ ]:
# For this to work you have to change the runtime type to GPU

In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from scipy.sparse import hstack

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
large_data = pd.read_csv(DATA_DIR + AMAZON_DATA_TRAIN)
data = large_data.sample(frac=0.01, random_state=101)
data.columns = ['Sentiment', 'Review Title', 'Text']
data.drop('Review Title', axis=1, inplace=True)

data['Length'] = data['Text'].apply(len)
data['Word Count'] = data['Text'].apply(lambda x: len(x.split()))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [ ]:
def get_bert_embeddings(texts, batch_size=32):
    bert_model.eval()
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Move to GPU if available

        with torch.no_grad():
            outputs = bert_model(**inputs)

        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move back to CPU
        all_embeddings.append(cls_embeddings)

    return np.vstack(all_embeddings)

texts = data['Text'].tolist()
bert_features = get_bert_embeddings(texts, batch_size=32)

custom_features = data[['Length', 'Word Count']].values

combined_features = np.hstack([bert_features, custom_features])

x_train, x_test, y_train, y_test = train_test_split(combined_features, data['Sentiment'], test_size=0.3, random_state=101)

svm = LinearSVC()
clf = CalibratedClassifierCV(svm)
clf.fit(x_train, y_train)

y_predict = clf.predict(x_test)

print("############# Classification Report ##############")
print(classification_report(y_test, y_predict))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test, y_predict))
print("##################################################")

############# Classification Report ##############
              precision    recall  f1-score   support

           1       0.86      0.88      0.87      5313
           2       0.88      0.86      0.87      5487

    accuracy                           0.87     10800
   macro avg       0.87      0.87      0.87     10800
weighted avg       0.87      0.87      0.87     10800

##################################################
############# Confusion Matrix ##############
[[4676  637]
 [ 757 4730]]
##################################################


# RoBERTa Embedding Features

In [ ]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from scipy.sparse import hstack

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
large_data = pd.read_csv(DATA_DIR + AMAZON_DATA_TRAIN)
data = large_data.sample(frac=0.01, random_state=101)
data.columns = ['Sentiment', 'Review Title', 'Text']
data.drop('Review Title', axis=1, inplace=True)

data['Length'] = data['Text'].apply(len)
data['Word Count'] = data['Text'].apply(lambda x: len(x.split()))

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaModel.from_pretrained('roberta-base').to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def get_roberta_embeddings(texts, batch_size=32):
    roberta_model.eval()
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = roberta_model(**inputs)

        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(cls_embeddings)

    return np.vstack(all_embeddings)

texts = data['Text'].tolist()
roberta_features = get_roberta_embeddings(texts, batch_size=32)

custom_features = data[['Length', 'Word Count']].values
combined_features = np.hstack([roberta_features, custom_features])

x_train, x_test, y_train, y_test = train_test_split(combined_features, data['Sentiment'], test_size=0.3, random_state=101)

svm = LinearSVC()
clf = CalibratedClassifierCV(svm)
clf.fit(x_train, y_train)

y_predict = clf.predict(x_test)

print("############# Classification Report ##############")
print(classification_report(y_test, y_predict))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test, y_predict))
print("##################################################")

############# Classification Report ##############
              precision    recall  f1-score   support

           1       0.91      0.91      0.91      5313
           2       0.92      0.91      0.91      5487

    accuracy                           0.91     10800
   macro avg       0.91      0.91      0.91     10800
weighted avg       0.91      0.91      0.91     10800

##################################################
############# Confusion Matrix ##############
[[4861  452]
 [ 482 5005]]
##################################################


In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_XGBC = le.fit_transform(y_train)
y_test_XGBC = le.fit_transform(y_test)


XGBC = XGBClassifier(objective="binary:logistic", random_state=42)
clf_XGBC = CalibratedClassifierCV(XGBC)
clf_XGBC.fit(x_train, y_train_XGBC)

y_predict_XGBC = clf_XGBC.predict(x_test)

print("############# Classification Report ##############")
print(classification_report(y_test_XGBC, y_predict_XGBC))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test_XGBC, y_predict_XGBC))
print("##################################################")

############# Classification Report ##############
              precision    recall  f1-score   support

           0       0.89      0.90      0.90      5313
           1       0.90      0.89      0.90      5487

    accuracy                           0.90     10800
   macro avg       0.90      0.90      0.90     10800
weighted avg       0.90      0.90      0.90     10800

##################################################
############# Confusion Matrix ##############
[[4799  514]
 [ 600 4887]]
##################################################


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
clf_RF = CalibratedClassifierCV(RF)
clf_RF.fit(x_train, y_train)

y_predict_RF = clf_RF.predict(x_test)

print("############# Classification Report ##############")
print(classification_report(y_test, y_predict_RF))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test, y_predict_RF))
print("##################################################")

############# Classification Report ##############
              precision    recall  f1-score   support

           1       0.86      0.87      0.86      5313
           2       0.87      0.86      0.87      5487

    accuracy                           0.87     10800
   macro avg       0.87      0.87      0.87     10800
weighted avg       0.87      0.87      0.87     10800

##################################################
############# Confusion Matrix ##############
[[4635  678]
 [ 772 4715]]
##################################################


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
clf_KNN = CalibratedClassifierCV(KNN)
clf_KNN.fit(x_train, y_train)

y_predict_KNN = clf_KNN.predict(x_test)

print("############# Classification Report ##############")
print(classification_report(y_test, y_predict_KNN))
print("##################################################")

print("############# Confusion Matrix ##############")
print(confusion_matrix(y_test, y_predict_KNN))
print("##################################################")

############# Classification Report ##############
              precision    recall  f1-score   support

           1       0.53      0.53      0.53      5313
           2       0.55      0.55      0.55      5487

    accuracy                           0.54     10800
   macro avg       0.54      0.54      0.54     10800
weighted avg       0.54      0.54      0.54     10800

##################################################
############# Confusion Matrix ##############
[[2806 2507]
 [2484 3003]]
##################################################
